# Spaceship Titanic NN Model 🌠

Hello Kaggle; 
My goal is to provide the best single **Neuronal Network Model** for the **Space Ship Titanic** competition...
I follow this typical strategy in majority of my Notebook and works quite well for me so I want it to share it.

## Strategy

* Installing Libraries.
* Importing Requiered Libraries
* Setting Notebook Parameters
* Loading the Datasets from CSV to Pandas Dataset
* Exploring the Datasets
* Creating Additional Features
* Pre-Processing
* Feature Selection
* Advance Feature Creation (Clustering)
* Selecting Features for Model Developmetn
* Developing a DNN Model
* Training a Model Utilizing a CV Loop
* Generating Finala Predictions


## File and Data Field Descriptions

train.csv - Personal records for about two-thirds (~8700) of the passengers, to be used as training data.
* **PassengerId** - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
* **HomePlanet** - The planet the passenger departed from, typically their planet of permanent residence.
* **CryoSleep** - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
* **Cabin** - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
* **Destination** - The planet the passenger will be debarking to.
* **Age** - The age of the passenger.
* **VIP** - Whether the passenger has paid for special VIP service during the voyage.
* **RoomService**, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
* **Name** - The first and last names of the passenger.
* **Transported** - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

test.csv - Personal records for the remaining one-third (~4300) of the passengers, to be used as test data. Your task is to predict the value of Transported for the passengers in this set.

sample_submission.csv - A submission file in the correct format.
* **PassengerId** - Id for each passenger in the test set.
* **Transported** - The target. For each passenger, predict either True or False.


## Baseline Model
Hello, here is the location of a basaline model for the competition if you are interested in the modeling component...

https://www.kaggle.com/code/cv13j0/spaceship-my-starter-model


**Last Updates**

Worked on the notebook structure and format.

# 1.0 Installing Required Libraries

In [1]:
%%capture
!git clone https://github.com/analokmaus/kuma_utils.git

In [2]:
%%time
import os
import sys
sys.path.append("kuma_utils/")
from kuma_utils.preprocessing.imputer import LGBMImputer

CPU times: user 1.42 s, sys: 376 ms, total: 1.8 s
Wall time: 3.24 s


In [3]:
%%capture
# Install Gokinjo...
!pip install gokinjo

---

# 2.0 Importing Libraries for the Model

In [4]:
%%time
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv
CPU times: user 1.61 ms, sys: 0 ns, total: 1.61 ms
Wall time: 3.01 ms


In [5]:
%%capture
from sklearn.preprocessing import LabelEncoder
from gokinjo import knn_kfold_extract
from gokinjo import knn_extract

In [6]:
%%time
from sklearn.preprocessing import LabelEncoder 

CPU times: user 7 µs, sys: 1e+03 ns, total: 8 µs
Wall time: 11.2 µs


---

# 3.0 Seeting Notebook Parameters...

In [7]:
%%time
# I like to disable my Notebook Warnings.
import warnings
warnings.filterwarnings('ignore')

CPU times: user 63 µs, sys: 10 µs, total: 73 µs
Wall time: 76.1 µs


In [8]:
%%time
# Notebook Configuration...

# Amount of data we want to load into the Model...
DATA_ROWS = None
# Dataframe, the amount of rows and cols to visualize...
NROWS = 100
NCOLS = 8
# Main data location path...
BASE_PATH = '...'

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.34 µs


In [9]:
%%time
# Configure notebook display settings to only use 2 decimal places, tables look nicer.
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', NCOLS) 
pd.set_option('display.max_rows', NROWS)

CPU times: user 133 µs, sys: 0 ns, total: 133 µs
Wall time: 136 µs


---

# 4.0 Loading Information from CSV...

In [10]:
%%time
trn_data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
tst_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

sub = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')

CPU times: user 31.8 ms, sys: 13.1 ms, total: 44.9 ms
Wall time: 76.3 ms


---

# 5.0 Exploring the Information Available

In [11]:
%%time
# Explore the shape of the DataFrame...
trn_data.shape

CPU times: user 28 µs, sys: 5 µs, total: 33 µs
Wall time: 36 µs


(8693, 14)

In [12]:
%%time
# Display the first few rows of the DataFrame...
trn_data.head()

CPU times: user 0 ns, sys: 212 µs, total: 212 µs
Wall time: 218 µs


,PassengerId,HomePlanet,CryoSleep,Cabin,...,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,...,0.00,0.00,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,...,549.00,44.00,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,...,"6,715.00",49.00,Altark Susent,False
3,0003_02,Europa,False,A/0/S,...,"3,329.00",193.00,Solam Susent,False
4,0004_01,Earth,False,F/1/S,...,565.00,2.00,Willy Santantines,True


In [13]:
%%time
# Display the information from the dataset...
trn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB
CPU times: user 8.16 ms, sys: 4.06 ms, total: 12.2 ms
Wall time: 20.5 ms


In [14]:
%%time
# Checking for empty or NaN values in the dataset by variable...
trn_data.isnull().sum()

CPU times: user 5.12 ms, sys: 8 µs, total: 5.13 ms
Wall time: 4.75 ms


PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

---

# 6.0 Feature Engineering...

## 6.1 Filling NaNs by Using EDA Insights (Age)

In [15]:
%%time
# Filling NaNs Based on Feature Engineering...
def fill_nans_by_age(df, age_limit = 13):
    df['RoomService'] = np.where(df['Age'] <= age_limit, 0, df['RoomService'])
    df['FoodCourt'] = np.where(df['Age'] <= age_limit, 0, df['FoodCourt'])
    df['ShoppingMall'] = np.where(df['Age'] <= age_limit, 0, df['ShoppingMall'])
    df['Spa'] = np.where(df['Age'] <= age_limit, 0, df['Spa'])
    df['VRDeck'] = np.where(df['Age'] <= age_limit, 0, df['VRDeck'])
    
    return df

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


In [16]:
%%time
trn_data =  fill_nans_by_age(trn_data)
tst_data =  fill_nans_by_age(tst_data)

CPU times: user 5.27 ms, sys: 199 µs, total: 5.47 ms
Wall time: 5.86 ms


---

## 6.2 Filling NaNs by Using EDA Insights (CryoSleep)

In [17]:
%%time
# Filling NaNs Based on Feature Engineering...
def fill_nans_by_cryo(df, age_limit = 13):
    df['RoomService'] = np.where(df['CryoSleep'] == True, 0, df['RoomService'])
    df['FoodCourt'] = np.where(df['CryoSleep'] == True, 0, df['FoodCourt'])
    df['ShoppingMall'] = np.where(df['CryoSleep'] == True, 0, df['ShoppingMall'])
    df['Spa'] = np.where(df['CryoSleep'] == True, 0, df['Spa'])
    df['VRDeck'] = np.where(df['CryoSleep'] == True, 0, df['VRDeck'])
    
    return df

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


In [18]:
%%time
trn_data =  fill_nans_by_cryo(trn_data)
tst_data =  fill_nans_by_cryo(tst_data)

CPU times: user 7.92 ms, sys: 982 µs, total: 8.9 ms
Wall time: 8.81 ms


---

## 6.3 Creating Age Groups Using EDA Insights (Age)

In [19]:
%%time
def age_groups(df, age_limit = 13):
    df['AgeGroup'] = np.where(df['Age'] < age_limit, 0, 1)
    return df

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


In [20]:
%%time
trn_data =  age_groups(trn_data)
tst_data =  age_groups(tst_data)

CPU times: user 2.15 ms, sys: 21 µs, total: 2.17 ms
Wall time: 2.12 ms


---

## 6.7 Extracting Deck, Cabin Number and Side

In [21]:
%%time
def cabin_separation(df):
    '''
    Split the Cabin name into Deck, Number and Side
    
    '''
    
    df['CabinDeck'] = df['Cabin'].str.split('/', expand=True)[0]
    df['CabinNum']  = df['Cabin'].str.split('/', expand=True)[1]
    df['CabinSide'] = df['Cabin'].str.split('/', expand=True)[2]
    
    df.drop(columns = ['Cabin'], inplace = True)
    return df

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.87 µs


In [22]:
%%time
trn_data = cabin_separation(trn_data)
tst_data = cabin_separation(tst_data)

CPU times: user 57.8 ms, sys: 2.88 ms, total: 60.7 ms
Wall time: 60 ms


---

## 6.8 Extracting Family Name and Name

In [23]:
%%time
def name_ext(df):
    '''
    Split the Name of the passenger into First and Family...
    
    '''
    
    df['FirstName'] = df['Name'].str.split(' ', expand=True)[0]
    df['FamilyName'] = df['Name'].str.split(' ', expand=True)[1]
    df.drop(columns = ['Name'], inplace = True)
    return df

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


In [24]:
%%time
trn_data = name_ext(trn_data)
tst_data = name_ext(tst_data)

CPU times: user 42.6 ms, sys: 1.86 ms, total: 44.4 ms
Wall time: 43.7 ms


---

## 6.9 Creating Age Groups, Based on EDA

In [25]:
%%time
def age_groups(df, age_limit = 13):
    df['AgeGroup'] = np.where(df['Age'] < age_limit, 0, 1)
    return df

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.34 µs


In [26]:
%%time
trn_data =  age_groups(trn_data)
tst_data =  age_groups(tst_data)

CPU times: user 2.7 ms, sys: 109 µs, total: 2.81 ms
Wall time: 2.36 ms


---

## 6.10 Extracting Group

In [27]:
def extract_group(df):
    '''
    '''
    df['TravelGroup'] =  df['PassengerId'].str.split('_', expand = True)[0]
    df['TravelGroupPos'] =  df['PassengerId'].str.split('_', expand = True)[1]
    return df

In [28]:
%%time
trn_data = extract_group(trn_data)
tst_data = extract_group(tst_data)

CPU times: user 167 ms, sys: 22.1 ms, total: 189 ms
Wall time: 190 ms


## 6.11 Imputing Using LightGBM

In [29]:
trn_data.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Transported', 'AgeGroup', 'CabinDeck', 'CabinNum', 'CabinSide',
       'FirstName', 'FamilyName', 'TravelGroup', 'TravelGroupPos'],
      dtype='object')

In [30]:
cols = [
        #'HomePlanet',
        #'CryoSleep',
        #'Destination',
        'Age',
        #'VIP',
        'RoomService',
        'FoodCourt',
        'ShoppingMall',
        'Spa',
        'VRDeck',
        #'AgeGroup',
        #'CabinDeck',
        #'CabinNum',
        #'CabinSide',
        #'FirstName',
        #'FamilyName',
        #'TravelGroup'
       ]

object_cols = [
               #'HomePlanet',
               #'CryoSleep',
               #'Destination',
               #'VIP',
               #'CabinDeck',
               #'CabinNum',
               #'CabinSide',
               #'FirstName',
               #'FamilyName',
               #'TravelGroup'
              ]

In [31]:
trn_data[cols].isnull().sum()

Age             179
RoomService     106
FoodCourt       106
ShoppingMall    103
Spa             109
VRDeck          106
dtype: int64

In [32]:
trn_data['VIP']

0       False
1       False
2        True
3       False
4       False
        ...  
8688     True
8689    False
8690    False
8691    False
8692    False
Name: VIP, Length: 8693, dtype: object

In [33]:
%%time
object_cols = ['HomePlanet','CryoSleep','Cabin','Destination','VIP','Name']

# Create a fill missing values function using the Simple Imputer
def ml_imputer(df, cols, object_cols):
    '''
    
    '''
    tmp = df.copy(deep = True)
    lgbm_imputer = LGBMImputer(cat_features = object_cols, n_iter = 50)
    tmp[cols] = lgbm_imputer.fit_transform(tmp[cols])
    print('...........', '\n')
    return tmp

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.63 µs


In [34]:
%%time
trn_data = ml_imputer(trn_data, cols, object_cols)
tst_data = ml_imputer(tst_data, cols, object_cols)

  0%|          | 0/6 [00:00<?, ?it/s]

........... 



  0%|          | 0/6 [00:00<?, ?it/s]

........... 

CPU times: user 1.72 s, sys: 17.3 ms, total: 1.74 s
Wall time: 932 ms


In [35]:
trn_data.isnull().sum()

PassengerId         0
HomePlanet        201
CryoSleep         217
Destination       182
Age                 0
VIP               203
RoomService         0
FoodCourt           0
ShoppingMall        0
Spa                 0
VRDeck              0
Transported         0
AgeGroup            0
CabinDeck         199
CabinNum          199
CabinSide         199
FirstName         200
FamilyName        200
TravelGroup         0
TravelGroupPos      0
dtype: int64

---

## 6.4 Filling NaNs by Mode and Mean (Using Groups)

In [36]:
trn_data[trn_data['TravelGroup'] == '2926']

,PassengerId,HomePlanet,CryoSleep,Destination,...,FirstName,FamilyName,TravelGroup,TravelGroupPos
2726,2926_01,Mars,False,TRAPPIST-1e,...,Kad,Homin,2926,01


In [37]:

people_per_group = trn_data.groupby('TravelGroup')['CabinDeck'].nunique().reset_index()
people_per_group.sort_values('CabinDeck', ascending=False)

,TravelGroup,CabinDeck
2052,3060,3
623,0938,3
4370,6468,3
4278,6334,3
2611,3911,3
...,...,...
1964,2926,0
1950,2900,0
4995,7429,0
349,0525,0


In [38]:
people_per_group.max()

TravelGroup    9280
CabinDeck         3
dtype: object

In [39]:
%%time
def fill_missing(df, group = 'TravelGroup',):
    '''
    Fill NaNs values or with mean or most commond value...
    
    '''
    
    tmp = df.copy()
    tmp = tpm[tmp[group].ising()]
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    
    numeric_tmp = df.select_dtypes(include = numerics)
    categ_tmp = df.select_dtypes(exclude = numerics)
    

    for col in numeric_tmp.columns:
        print(col)
        #df[col] = df[col].fillna(value = df[col].mean())
        df[col] = df[col].fillna(df.groupby(group)[col].transform('mean'))
        
    for col in categ_tmp.columns:
        print(col)
        #df[col] = df[col].fillna(value = df[col].mode()[0])
        #df[col] = df[col].fillna(df.groupby(group)[col].transform(lambda S: S.mode()[0]))
        df[col] = (df.groupby(group)[col].transform(lambda x: x.value_counts().idxmax()))
    print('...')
    
    return df

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


In [40]:
%%time
#trn_data =  fill_missing(trn_data)
#tst_data =  fill_missing(tst_data)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


## 6.4 Filling NaNs by Mode and Mean

In [41]:
%%time
def fill_missing(df):
    '''
    Fill NaNs values or with mean or most commond value...
    
    '''
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    
    numeric_tmp = df.select_dtypes(include = numerics)
    categ_tmp = df.select_dtypes(exclude = numerics)

    for col in numeric_tmp.columns:
        print(col)
        df[col] = df[col].fillna(value = df[col].mean())
        
    for col in categ_tmp.columns:
        print(col)
        df[col] = df[col].fillna(value = df[col].mode()[0])
        
    print('...')
    
    return df

CPU times: user 8 µs, sys: 1e+03 ns, total: 9 µs
Wall time: 12.6 µs


In [42]:
%%time
trn_data =  fill_missing(trn_data)
tst_data =  fill_missing(tst_data)

Age
RoomService
FoodCourt
ShoppingMall
Spa
VRDeck
AgeGroup
PassengerId
HomePlanet
CryoSleep
Destination
VIP
Transported
CabinDeck
CabinNum
CabinSide
FirstName
FamilyName
TravelGroup
TravelGroupPos
...
Age
RoomService
FoodCourt
ShoppingMall
Spa
VRDeck
AgeGroup
PassengerId
HomePlanet
CryoSleep
Destination
VIP
CabinDeck
CabinNum
CabinSide
FirstName
FamilyName
TravelGroup
TravelGroupPos
...
CPU times: user 77.4 ms, sys: 2.1 ms, total: 79.5 ms
Wall time: 72.2 ms


In [43]:
trn_data.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep         0
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
AgeGroup          0
CabinDeck         0
CabinNum          0
CabinSide         0
FirstName         0
FamilyName        0
TravelGroup       0
TravelGroupPos    0
dtype: int64

## 6.5 Calculating Total Expended in the Ship

In [44]:
%%time
def total_billed(df):
    '''
    Calculates total amount billed in the trip to the passenger... 
    Args:
    Returns:
    
    '''
    
    df['TotalBilled'] = df['RoomService'] + df['FoodCourt'] + df['ShoppingMall'] + df['Spa'] + df['VRDeck']
    return df

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.34 µs


In [45]:
%%time
trn_data = total_billed(trn_data)
tst_data = total_billed(tst_data)

CPU times: user 5.08 ms, sys: 134 µs, total: 5.21 ms
Wall time: 4.7 ms


## 6.6 Filling NaNs by Using EDA Insights (TotalBilled)

In [46]:
%%time
def fill_nans_by_totalspend(df):
    df['CryoSleep'] = np.where(df['TotalBilled'] > 0, False, df['CryoSleep'])
    return df

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 11.2 µs


In [47]:
%%time
trn_data =  fill_nans_by_totalspend(trn_data)
tst_data =  fill_nans_by_totalspend(tst_data)

CPU times: user 2.17 ms, sys: 18 µs, total: 2.18 ms
Wall time: 2.11 ms


---

## 6.11 Calculating Aggregated Features, Based on Cabin Deck

In [48]:
%%time
Weltiest_Deck = trn_data.groupby('CabinDeck').aggregate({'TotalBilled': 'sum', 'Transported': 'sum', 'CryoSleep': 'sum', 'PassengerId': 'size'}).reset_index()
Weltiest_Deck['AvgSpended'] = Weltiest_Deck['TotalBilled'] / Weltiest_Deck['PassengerId']
Weltiest_Deck['TransportedPercentage'] = Weltiest_Deck['Transported'] / Weltiest_Deck['PassengerId']
Weltiest_Deck['CryoSleepPercentage'] = Weltiest_Deck['CryoSleep'] / Weltiest_Deck['PassengerId']
Weltiest_Deck = Weltiest_Deck.sort_values('AvgSpended', ascending = False)
Weltiest_Deck.head(10)

CPU times: user 7.73 ms, sys: 864 µs, total: 8.59 ms
Wall time: 8.03 ms


,CabinDeck,TotalBilled,Transported,CryoSleep,PassengerId,AvgSpended,TransportedPercentage,CryoSleepPercentage
7,T,"23,658.93",1,0,5,"4,731.79",0.20,0.00
2,C,"3,087,053.69",508,294,747,"4,132.60",0.68,0.39
0,A,"872,932.97",127,68,256,"3,409.89",0.50,0.27
1,B,"2,271,711.04",572,419,779,"2,916.19",0.73,0.54
3,D,"1,101,717.59",207,104,478,"2,304.85",0.43,0.22
4,E,"1,145,011.99",313,168,876,"1,307.09",0.36,0.19
5,F,"3,089,266.40",1329,634,2993,"1,032.16",0.44,0.21
6,G,"1,008,888.44",1321,1350,2559,394.25,0.52,0.53


In [49]:
%%time
trn_data = trn_data.merge(Weltiest_Deck[['CabinDeck', 'TransportedPercentage', 'AvgSpended']], how = 'left', on = ['CabinDeck'])
tst_data = tst_data.merge(Weltiest_Deck[['CabinDeck', 'TransportedPercentage', 'AvgSpended']], how = 'left', on = ['CabinDeck'])

CPU times: user 27 ms, sys: 476 µs, total: 27.4 ms
Wall time: 50.3 ms


---

## 6.12 Calulating the Number of Relatives, Using Family Name

In [50]:
%%time
trn_relatives = trn_data.groupby('FamilyName')['PassengerId'].count().reset_index()
tst_relatives = tst_data.groupby('FamilyName')['PassengerId'].count().reset_index()

CPU times: user 12.3 ms, sys: 0 ns, total: 12.3 ms
Wall time: 16 ms


In [51]:
%%time
trn_relatives = trn_relatives.rename(columns = {'PassengerId': 'NumRelatives'})
tst_relatives = tst_relatives.rename(columns = {'PassengerId': 'NumRelatives'})

CPU times: user 1.19 ms, sys: 23 µs, total: 1.21 ms
Wall time: 1.14 ms


In [52]:
%%time
trn_data = trn_data.merge(trn_relatives[['FamilyName', 'NumRelatives']], how = 'left', on = ['FamilyName'])
tst_data = tst_data.merge(tst_relatives[['FamilyName', 'NumRelatives']], how = 'left', on = ['FamilyName'])

CPU times: user 14.3 ms, sys: 146 µs, total: 14.4 ms
Wall time: 14.3 ms


---

## 6.13 Calulating the Number of People Traveling Together, Using Traveling Group**

In [53]:
%%time
trn_relatives = trn_data.groupby('TravelGroup')['PassengerId'].count().reset_index()
tst_relatives = tst_data.groupby('TravelGroup')['PassengerId'].count().reset_index()

CPU times: user 11.5 ms, sys: 45 µs, total: 11.5 ms
Wall time: 10.9 ms


In [54]:
%%time
trn_relatives = trn_relatives.rename(columns = {'PassengerId': 'GroupSize'})
tst_relatives = tst_relatives.rename(columns = {'PassengerId': 'GroupSize'})

CPU times: user 782 µs, sys: 981 µs, total: 1.76 ms
Wall time: 1.59 ms


In [55]:
%%time
trn_data = trn_data.merge(trn_relatives[['TravelGroup', 'GroupSize']], how = 'left', on = ['TravelGroup'])
tst_data = tst_data.merge(tst_relatives[['TravelGroup', 'GroupSize']], how = 'left', on = ['TravelGroup'])

CPU times: user 17 ms, sys: 1.01 ms, total: 18 ms
Wall time: 17.6 ms


---

# 7.0 Pre-Processing for Training

## 7.1 Separating the Fields by Type

In [56]:
trn_data.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Transported', 'AgeGroup', 'CabinDeck', 'CabinNum', 'CabinSide',
       'FirstName', 'FamilyName', 'TravelGroup', 'TravelGroupPos',
       'TotalBilled', 'TransportedPercentage', 'AvgSpended', 'NumRelatives',
       'GroupSize'],
      dtype='object')

In [57]:
%%time
# A list of the original variables from the dataset
numerical_features = [
                      'Age', 
                      'RoomService', 
                      'FoodCourt', 
                      'ShoppingMall', 
                      'Spa', 
                      'VRDeck', 
                      'TotalBilled',
                      'TravelGroupPos'
                     ]

categorical_features = [
                        #'Name',
                        'FirstName',
                        'FamilyName',
                        'CabinNum',
                        'TravelGroup',
                        'AgeGroup'
                       ]


categorical_features_onehot = [
                               'HomePlanet',
                               'CryoSleep',
                               #'Cabin',
                               'CabinDeck',
                               'CabinSide',
                               'Destination',
                               'VIP',
                               #'AgeGroup'
                               ]

target_feature = 'Transported'

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.15 µs


---

## 7.2 Encoding Categorical Variables

In [58]:
%%time

def encode_categorical(train_df, test_df, categ_feat = categorical_features):
    '''
    
    '''
    encoder_dict = {}
    
    concat_data = pd.concat([trn_data[categ_feat], tst_data[categ_feat]])
    
    for col in concat_data.columns:
        print('Encoding: ', col, '...')
        encoder = LabelEncoder()
        encoder.fit(concat_data[col])
        encoder_dict[col] = encoder

        train_df[col + '_Enc'] = encoder.transform(train_df[col])
        test_df[col + '_Enc'] = encoder.transform(test_df[col])
    
    train_df = train_df.drop(columns = categ_feat, axis = 1)
    test_df = test_df.drop(columns = categ_feat, axis = 1)

    return train_df, test_df

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.34 µs


In [59]:
%%time
trn_data, tst_data = encode_categorical(trn_data, tst_data, categorical_features)

Encoding:  FirstName ...
Encoding:  FamilyName ...
Encoding:  CabinNum ...
Encoding:  TravelGroup ...
Encoding:  AgeGroup ...
CPU times: user 71.6 ms, sys: 3.14 ms, total: 74.8 ms
Wall time: 72.4 ms


---

## 7.3 One Hot Encoding Categorical Variables

In [60]:
%%time
def one_hot(df, one_hot_categ):
    for col in one_hot_categ:
        tmp = pd.get_dummies(df[col], prefix = col)
        df = pd.concat([df, tmp], axis = 1)
    df = df.drop(columns = one_hot_categ)
    return df

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


In [61]:
%%time
trn_data = one_hot(trn_data, categorical_features_onehot) 
tst_data = one_hot(tst_data, categorical_features_onehot) 

CPU times: user 30.5 ms, sys: 1.03 ms, total: 31.5 ms
Wall time: 34 ms


---

# 8.0 Feature Selection for Baseline Model

In [62]:
%%time
remove = ['PassengerId', 
          'Route', 
          'FirstName_Enc', 
          #'CabinNum_Enc', 
          'Transported',
          'Cabin',
          'TransportedPercentage',
          #'IsKid', 
          #'IsAdult', 
          #'IsOlder'
          #'RoomService',
          #'FoodCourt',
          #'ShoppingMall',
          #'Spa',
          #'VRDeck',
         ]
features = [feat for feat in trn_data.columns if feat not in remove]

CPU times: user 36 µs, sys: 4 µs, total: 40 µs
Wall time: 44.8 µs


In [63]:
%%time
features

CPU times: user 0 ns, sys: 3 µs, total: 3 µs
Wall time: 6.44 µs


['Age',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'TravelGroupPos',
 'TotalBilled',
 'AvgSpended',
 'NumRelatives',
 'GroupSize',
 'FamilyName_Enc',
 'CabinNum_Enc',
 'TravelGroup_Enc',
 'AgeGroup_Enc',
 'HomePlanet_Earth',
 'HomePlanet_Europa',
 'HomePlanet_Mars',
 'CryoSleep_False',
 'CryoSleep_True',
 'CabinDeck_A',
 'CabinDeck_B',
 'CabinDeck_C',
 'CabinDeck_D',
 'CabinDeck_E',
 'CabinDeck_F',
 'CabinDeck_G',
 'CabinDeck_T',
 'CabinSide_P',
 'CabinSide_S',
 'Destination_55 Cancri e',
 'Destination_PSO J318.5-22',
 'Destination_TRAPPIST-1e',
 'VIP_False',
 'VIP_True']

---

# 9.0 Advance Feature Engineering, KNN

In [64]:
%%time
# Convert X and y to Numpy arrays as library requirements
X_array = trn_data[features].to_numpy()
y_array = trn_data['Transported'].to_numpy()
X_test_array = tst_data[features].to_numpy()

CPU times: user 14.5 ms, sys: 1.94 ms, total: 16.5 ms
Wall time: 15.8 ms


In [65]:
K = 2

In [66]:
%%time
# It Takes almost  35min 21s for K = 2 and 50_000 rows...
# It Takes almost  17min 36s for K = 1 and 50_000 rows...
KNN_trn_features = knn_kfold_extract(X_array, y_array, k = K, normalize = 'standard')

CPU times: user 2.73 s, sys: 1.71 s, total: 4.43 s
Wall time: 2.64 s


In [67]:
%%time
KNN_trn_features

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.15 µs


array([[ 1.65931539,  3.93703875,  2.56817486,  5.56926636],
       [ 0.81391176,  2.22018929,  1.19468296,  2.51241985],
       [ 6.63154324, 13.80597716,  6.73864297, 14.0130557 ],
       ...,
       [ 2.51176767,  5.0865934 ,  2.05225096,  4.18929819],
       [ 2.51876769,  5.19856235,  4.071007  ,  8.68026553],
       [ 2.3665048 ,  5.56743923,  2.06323883,  4.7418424 ]])

In [68]:
%%time
knn_cols = ['KNN_K1_01',
            'KNN_K1_02',
            'KNN_K2_01',
            'KNN_K2_02']

KNN_feat = pd.DataFrame(KNN_trn_features, columns = knn_cols)
KNN_feat = pd.DataFrame(KNN_trn_features, columns = knn_cols).set_index(trn_data.index)

CPU times: user 2.14 ms, sys: 29 µs, total: 2.17 ms
Wall time: 1.72 ms


In [69]:
%%time
trn_data = pd.concat([trn_data, KNN_feat], axis = 1)
trn_data.head()

CPU times: user 4.5 ms, sys: 112 µs, total: 4.61 ms
Wall time: 4.16 ms


,PassengerId,Age,RoomService,FoodCourt,...,KNN_K1_01,KNN_K1_02,KNN_K2_01,KNN_K2_02
0,0001_01,39.00,0.00,0.00,...,1.66,3.94,2.57,5.57
1,0002_01,24.00,109.00,9.00,...,0.81,2.22,1.19,2.51
2,0003_01,58.00,43.00,"3,576.00",...,6.63,13.81,6.74,14.01
3,0003_02,33.00,0.00,"1,283.00",...,2.39,6.15,3.93,8.01
4,0004_01,16.00,303.00,70.00,...,0.94,2.25,1.12,2.28


In [70]:
%%time
KNN_tst_features = knn_extract(X_array, y_array, X_test_array, k = K, normalize = 'standard')
KNN_feat = pd.DataFrame(KNN_tst_features, columns = knn_cols).set_index(tst_data.index)

tst_data = pd.concat([tst_data, KNN_feat], axis = 1)
tst_data.head()

CPU times: user 1.2 s, sys: 649 ms, total: 1.85 s
Wall time: 1.08 s


,PassengerId,Age,RoomService,FoodCourt,...,KNN_K1_01,KNN_K1_02,KNN_K2_01,KNN_K2_02
0,0013_01,27.00,0.00,0.00,...,0.47,1.09,0.74,1.54
1,0018_01,19.00,0.00,9.00,...,1.57,3.28,2.63,5.38
2,0019_01,31.00,0.00,0.00,...,4.74,9.73,1.22,2.61
3,0021_01,38.00,0.00,"6,652.00",...,4.32,8.86,1.62,4.01
4,0023_01,20.00,10.00,0.00,...,0.78,1.67,0.89,1.87


---

# 10.0 Selection of Features for Training Stage

In [71]:
%%time
remove = ['PassengerId', 
          'Route', 
          'FirstName_Enc', 
          'CabinNum_Enc', 
          'Transported',
          'Cabin',
          'TransportedPercentage',
          #'IsKid', 
          #'IsAdult', 
          #'IsOlder'
          #'RoomService',
          #'FoodCourt',
          #'ShoppingMall',
          #'Spa',
          #'VRDeck',
          'KNN_K2_02',
          'KNN_K2_01',
         ]
features = [feat for feat in trn_data.columns if feat not in remove]

CPU times: user 34 µs, sys: 10 µs, total: 44 µs
Wall time: 47.4 µs


In [72]:
%%time
features

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.68 µs


['Age',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'TravelGroupPos',
 'TotalBilled',
 'AvgSpended',
 'NumRelatives',
 'GroupSize',
 'FamilyName_Enc',
 'TravelGroup_Enc',
 'AgeGroup_Enc',
 'HomePlanet_Earth',
 'HomePlanet_Europa',
 'HomePlanet_Mars',
 'CryoSleep_False',
 'CryoSleep_True',
 'CabinDeck_A',
 'CabinDeck_B',
 'CabinDeck_C',
 'CabinDeck_D',
 'CabinDeck_E',
 'CabinDeck_F',
 'CabinDeck_G',
 'CabinDeck_T',
 'CabinSide_P',
 'CabinSide_S',
 'Destination_55 Cancri e',
 'Destination_PSO J318.5-22',
 'Destination_TRAPPIST-1e',
 'VIP_False',
 'VIP_True',
 'KNN_K1_01',
 'KNN_K1_02']

In [73]:
features = ['Age',
            'RoomService',
            'FoodCourt',
            'ShoppingMall',
            'Spa',
            'VRDeck',
            'TravelGroupPos',
            'TotalBilled',
            #'AvgSpended',
            #'NumRelatives',
            #'GroupSize',
            'FamilyName_Enc',
            'TravelGroup_Enc',
            #'AgeGroup_Enc',
            'HomePlanet_Earth',
            'HomePlanet_Europa',
            'HomePlanet_Mars',
            'CryoSleep_False',
            'CryoSleep_True',
            'CabinDeck_A',
            'CabinDeck_B',
            'CabinDeck_C',
            'CabinDeck_D',
            'CabinDeck_E',
            'CabinDeck_F',
            'CabinDeck_G',
            'CabinDeck_T',
            'CabinSide_P',
            'CabinSide_S',
            'Destination_55 Cancri e',
            'Destination_PSO J318.5-22',
            'Destination_TRAPPIST-1e',
            'VIP_False',
            'VIP_True',
            #'AgeGroup_0',
            #'AgeGroup_1',
            #'AgeGroup_0',
            #'AgeGroup_1',
            'KNN_K1_01',
            #'KNN_K1_02'
]

---

# 11.0 Building a Neuronal Network Model...

## 11.1 Importing all the required libraries...

In [74]:
%%time
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, InputLayer, Add, BatchNormalization, Dropout, Concatenate

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import random

from sklearn.model_selection import KFold, StratifiedKFold 
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
import datetime
import math

CPU times: user 1.76 s, sys: 630 ms, total: 2.39 s
Wall time: 5.05 s


---

## 11.2 Defining the Model Architecture...

In [75]:
%%time
def nn_model_one():
    '''
    '''
    
    activation_func = 'swish'
    inputs = Input(shape = (len(features)))
    
    x = Dense(1024, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(inputs)
    
    x = BatchNormalization()(x)
    
    x = Dense(256, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(x)
    
    x = BatchNormalization()(x)

    x = Dense(128, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(x)
    
    x = BatchNormalization()(x)

    x = Dense(1 , 
              #use_bias  = True, 
              #kernel_regularizer = tf.keras.regularizers.l2(30e-6),
              activation = 'sigmoid')(x)
    
    model = Model(inputs, x)
    
    return model

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.58 µs


In [76]:
%%time
def nn_model_two():
    '''
    '''

    dropout_value = 0.025
    
    activation_func = 'swish'
    inputs = Input(shape = (len(features)))
    
    x = Dense(1024, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(inputs)
    
    x = BatchNormalization()(x)
    x = Dropout(dropout_value)(x)
    
    x = Dense(256, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(x)
    
    x = BatchNormalization()(x)
    x = Dropout(dropout_value)(x)

    x = Dense(128, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(x)
    
    x = BatchNormalization()(x)
    x = Dropout(dropout_value)(x)
    
    x = Dense(8, 
          #use_bias  = True, 
          kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
          activation = activation_func)(x)
    
    x = BatchNormalization()(x)
    x = Dropout(dropout_value)(x)
    
    x = Dense(1 , 
              #use_bias  = True, 
              #kernel_regularizer = tf.keras.regularizers.l2(30e-6),
              activation = 'sigmoid')(x)
    
    model = Model(inputs, x)
    
    return model

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


In [77]:
%%time
def nn_model_three():
    
    '''
    Function to define the Neuronal Network architecture...
    '''
    
    L2 = 65e-6
    dropout_value = 0.025
    activation_func = 'swish'
    inputs = Input(shape = (len(features)))
    
    x0 = Dense(1024, kernel_regularizer = tf.keras.regularizers.l2(L2), activation = activation_func)(inputs)
    x0 = BatchNormalization()(x0)
    x0 = Dropout(dropout_value)(x0)
    
    x1 = Dense(1024,  kernel_regularizer = tf.keras.regularizers.l2(L2), activation = activation_func)(x0)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(dropout_value)(x1)
    
    x1 = Dense(64,  kernel_regularizer = tf.keras.regularizers.l2(L2), activation = activation_func)(x1)
    x1 = Concatenate()([x1, x0])
    x1 = BatchNormalization()(x1)
    x1 = Dropout(dropout_value)(x1)
    
    x1 = Dense(16, kernel_regularizer = tf.keras.regularizers.l2(L2), activation = activation_func)(x1)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(dropout_value)(x1)
    
    x1 = Dense(1,  
               #kernel_regularizer = tf.keras.regularizers.l2(4e-4), 
               activation = 'sigmoid')(x1)
    
    model = Model(inputs, x1)
    
    return model

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.39 µs


---

## 11.3 Visualizing the Architecture Created...

In [78]:
%%time
architecture = nn_model_one()
architecture.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 31)]              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              32768     
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512   

2022-08-21 19:39:51.917546: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-21 19:39:51.923232: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-21 19:39:51.923976: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-21 19:39:51.926192: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

---

## 11.4 Defining Model Parameters for Training...

In [79]:
%%time
# Defining model parameters...
BATCH_SIZE         = 128
EPOCHS             = 250 
EPOCHS_COSINEDECAY = 250
DIAGRAMS           = True
USE_PLATEAU        = True
INFERENCE          = False
VERBOSE            = 0 
TARGET             = 'Transported'

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.11 µs


---

## 11.5 Defining Training Functions for the NN Model

In [80]:
 %%time
# Defining model training function...
def fit_model(X_train, y_train, X_val, y_val, run = 0):
    '''
    '''
    lr_start = 0.2
    start_time = datetime.datetime.now()
    
    #scaler = StandardScaler()
    #scaler = RobustScaler()
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)

    epochs = EPOCHS    
    lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 8, verbose = VERBOSE)
    es = EarlyStopping(monitor = 'val_loss',patience = 16, verbose = 1, mode = 'min', restore_best_weights = True)
    tm = tf.keras.callbacks.TerminateOnNaN()
    callbacks = [lr, es, tm]
    
    # Cosine Learning Rate Decay
    if USE_PLATEAU == False:
        epochs = EPOCHS_COSINEDECAY
        lr_end = 0.0002

        def cosine_decay(epoch):
            if epochs > 1:
                w = (1 + math.cos(epoch / (epochs - 1) * math.pi)) / 2
            else:
                w = 1
            return w * lr_start + (1 - w) * lr_end
        
        lr = LearningRateScheduler(cosine_decay, verbose = 0)
        callbacks = [lr, tm]
        
    model = nn_model_one()
    
    optimizer_func = tf.keras.optimizers.Adam(learning_rate = lr_start)
    loss_func = tf.keras.losses.BinaryCrossentropy()
    model.compile(optimizer = optimizer_func, loss = loss_func)
    
    X_val = scaler.transform(X_val)
    validation_data = (X_val, y_val)
    
    history = model.fit(X_train, 
                        y_train, 
                        validation_data = validation_data, 
                        epochs          = epochs,
                        verbose         = VERBOSE,
                        batch_size      = BATCH_SIZE,
                        shuffle         = True,
                        callbacks       = callbacks
                       )
    
    history_list.append(history.history)
    print(f'Training loss:{history_list[-1]["loss"][-1]:.3f}')
    callbacks, es, lr, tm, history = None, None, None, None, None
    
    
    y_val_pred = model.predict(X_val, batch_size = BATCH_SIZE, verbose = VERBOSE)
    y_val_pred = [1 if x > 0.5 else 0 for x in y_val_pred]
    
    score = accuracy_score(y_val, y_val_pred)
    print(f'Fold {run}.{fold} | {str(datetime.datetime.now() - start_time)[-12:-7]}'
          f'| ACC: {score:.5f}')
    
    score_list.append(score)
    
    tst_data_scaled = scaler.transform(tst_data[features])
    tst_pred = model.predict(tst_data_scaled)
    predictions.append(tst_pred)
    
    return model

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


---

# Training a XGBoost Classifier ...

# 12.0 Training the NN Model, Using a CV Loop

In [81]:
%%time
# Create empty lists to store NN information...
history_list = []
score_list   = []
predictions  = []

# Define kfolds for training purposes...
#kf = KFold(n_splits = 5)
kf = StratifiedKFold(n_splits = 5, random_state = 15, shuffle = True)

for fold, (trn_idx, val_idx) in enumerate(kf.split(trn_data, trn_data[TARGET])):
    X_train, X_val = trn_data.iloc[trn_idx][features], trn_data.iloc[val_idx][features]
    y_train, y_val = trn_data.iloc[trn_idx][TARGET], trn_data.iloc[val_idx][TARGET]
    
    fit_model(X_train, y_train, X_val, y_val)
    
print(f'OOF AUC: {np.mean(score_list):.5f}')

2022-08-21 19:39:57.219321: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Restoring model weights from the end of the best epoch.
Epoch 00085: early stopping
Training loss:0.396
Fold 0.0 | 00:26| ACC: 0.81541
Restoring model weights from the end of the best epoch.
Epoch 00067: early stopping
Training loss:0.376
Fold 0.1 | 00:19| ACC: 0.80276
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
Training loss:0.570
Fold 0.2 | 00:08| ACC: 0.77516
Restoring model weights from the end of the best epoch.
Epoch 00114: early stopping
Training loss:0.389
Fold 0.3 | 00:42| ACC: 0.81703
Restoring model weights from the end of the best epoch.
Epoch 00045: early stopping
Training loss:0.461
Fold 0.4 | 00:14| ACC: 0.78769
OOF AUC: 0.79961
CPU times: user 2min 2s, sys: 11.2 s, total: 2min 14s
Wall time: 1min 51s


In [82]:
# OOF AUC: 0.80375
# OOF AUC: 0.80433
# OOF AUC: 0.80536
# OOF AUC: 0.80778
# OOF AUC: 0.80812 ***
# OOF AUC: 0.79420
# OOF AUC: 0.79133

# OOF AUC: 0.79731
# OOF AUC: 0.79708

# OOF AUC: 0.79973 ***

# OOF AUC: 0.79777
# OOF AUC: 0.80387 ***
# OOF AUC: 0.80191 
# OOF AUC: 0.80364 ...

# OOF AUC: 0.80755 >>> 0.81061
# OOF AUC: 0.80421 >>> 0.80664
# OOF AUC: 0.80778 >>> 0.80664
# OOF AUC: 0.80904 >>>

---

# 13.0 Generating Predictions

In [83]:
%%time
# Populated the prediction on the submission dataset and creates an output file
sub['Transported'] = np.array(predictions).mean(axis = 0)
sub['Transported'] = np.where(sub['Transported'] > 0.5, True, False)
sub.to_csv('my_submission_051922.csv', index = False)


CPU times: user 8.56 ms, sys: 152 µs, total: 8.71 ms
Wall time: 8.99 ms


In [84]:
%%time
sub.head()

CPU times: user 127 µs, sys: 15 µs, total: 142 µs
Wall time: 147 µs


,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False


---